In [2]:
# import modules
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from pandas.io import sql
import datetime
import pandas as pd
import pandasql as ps

In [3]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
df1 = pd.read_csv('./dict/pub_info_unique_2col.txt', sep='\t')
df2 =  pd.read_csv('./dict/new_pubs_with_micro_level_field_assignment.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-15 14:29:48.662239
종료: 2024-02-15 14:30:22.923974


In [14]:
df1

,uid,pubyear2
0,WOS:000234963600047,2006
1,WOS:000239031100003,2006
2,WOS:000234737300014,2006
3,WOS:000572968700194,2006
4,WOS:000237241200012,2006
...,...,...
47423797,WOS:000883130700065,2022
47423798,WOS:000787659200001,2023
47423799,WOS:000815539500001,2022
47423800,WOS:000823752200014,2022


In [13]:
df2

,ut,cluster_id,core,ut_to_uid
0,202894100002,64,1,WOS:000202894100002
1,202894100003,230,1,WOS:000202894100003
2,202894100004,783,1,WOS:000202894100004
3,202894100005,1692,1,WOS:000202894100005
4,202894100006,64,1,WOS:000202894100006
...,...,...,...,...
24344006,951303100001,1972,1,WOS:000951303100001
24344007,951303400001,198,1,WOS:000951303400001
24344008,951775800001,995,1,WOS:000951775800001
24344009,951799500001,679,1,WOS:000951799500001


In [5]:
df1.info()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47423802 entries, 0 to 47423801
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   uid       object
 1   pubyear2  int64 
dtypes: int64(1), object(1)
memory usage: 723.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24344011 entries, 0 to 24344010
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   ut          int64 
 1   cluster_id  int64 
 2   core        int64 
 3   ut_to_uid   object
dtypes: int64(3), object(1)
memory usage: 742.9+ MB


In [6]:
# uid 컬럼의 고유한 값들의 수와 데이터 프레임의 행 수 비교
is_unique = len(df1['uid'].unique()) == len(df1)
print("UID 컬럼은 유일한가?", is_unique)

UID 컬럼은 유일한가? True


In [7]:
# uid 컬럼에서 중복된 값이 있는지 체크
has_duplicates = df1['uid'].duplicated().any()
print("UID 컬럼은 유일한가?", not has_duplicates)

UID 컬럼은 유일한가? True


In [8]:
# uid 컬럼의 고유한 값들의 수와 데이터 프레임의 행 수 비교
is_unique = len(df2['ut'].unique()) == len(df2)
print("UT 컬럼은 유일한가?", is_unique)

UT 컬럼은 유일한가? True


In [9]:
# 두 DataFrame의 uid 컬럼 데이터 타입 확인
print("Data type of df1['uid']:", df1['uid'].dtype)
print("Data type of df2['ut_to_uid']:", df2['ut_to_uid'].dtype)

Data type of df1['uid']: object
Data type of df2['ut_to_uid']: object


In [10]:

now = datetime.datetime.now() 
print('시작:', now)
# 두 DataFrame에서 uid 컬럼과 ut_to_uid 컬럼의 공통 값 찾기
# pd.merge 함수의 조인 조건 수정: df1.uid와 df2.ut_to_uid
common_uids = pd.merge(df1, df2, left_on='uid', right_on='ut_to_uid', how='inner')

# 매칭되는 uid 개수 카운트
matching_uid_count = common_uids['uid'].nunique()

print("Number of matching uids:", matching_uid_count)

now = datetime.datetime.now()
print('종료:', now)


시작: 2024-02-15 14:32:25.263508
Number of matching uids: 24339955
종료: 2024-02-15 14:33:44.623412


inner join하여 기준되는 테이블 만들기 + agg도 함께 만들기
agg_3.txt : uid, cluster_id, pubyear
agg_4.txt : uid, cluster_id, core, pubyear

In [16]:
common_uids
# Save DataFrame to txt
common_uids.to_csv('./mid/inner_join_p_db.txt', sep='\t', index=False)

In [17]:
now = datetime.datetime.now() 
print('시작:', now)

# 필요 컬럼만 추출
agg3 = common_uids[['uid', 'cluster_id', 'pubyear2']]
agg4 = common_uids[['uid', 'cluster_id', 'core', 'pubyear2']]

 

# Save DataFrame to txt
agg3.to_csv('./mid/agg3.txt', sep='\t', index=False)
agg4.to_csv('./mid/agg4.txt', sep='\t', index=False)

now = datetime.datetime.now()
print('종료:', now)



시작: 2024-02-15 17:29:40.528419
종료: 2024-02-15 17:30:58.993784


In [19]:
agg3


,uid,cluster_id,pubyear2
0,WOS:000239031100003,1333,2006
1,WOS:000234737300014,846,2006
2,WOS:000241844400003,997,2006
3,WOS:000417059300001,151,2006
4,WOS:000235792300010,210,2006
...,...,...,...
24339950,WOS:000750113400001,2719,2021
24339951,WOS:000839344100002,1636,2022
24339952,WOS:000798801800001,3584,2021
24339953,WOS:000763296900001,2034,2021


In [20]:
agg4

,uid,cluster_id,core,pubyear2
0,WOS:000239031100003,1333,1,2006
1,WOS:000234737300014,846,1,2006
2,WOS:000241844400003,997,0,2006
3,WOS:000417059300001,151,1,2006
4,WOS:000235792300010,210,1,2006
...,...,...,...,...
24339950,WOS:000750113400001,2719,0,2021
24339951,WOS:000839344100002,1636,1,2022
24339952,WOS:000798801800001,3584,0,2021
24339953,WOS:000763296900001,2034,0,2021


agg4에 n_clu 컬럼 생성 작업

In [21]:
# cluster_id 별로 개수를 세어 n_clu 컬럼을 생성
agg4['n_clu'] = agg4.groupby('cluster_id')['cluster_id'].transform('count')

C:\Users\User\AppData\Local\Temp\ipykernel_6864\2660720169.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agg4['n_clu'] = agg4.groupby('cluster_id')['cluster_id'].transform('count')


In [26]:
agg4.to_csv('./mid/agg5.txt', sep='\t', index=False)

깊은 복사 수행 
-> 이렇게 하면 new_agg4에 변경을 가해도 agg4는 영향을 받지 않음

In [27]:
agg5 = agg4.copy()

In [28]:
agg5

,uid,cluster_id,core,pubyear2,n_clu
0,WOS:000239031100003,1333,1,2006,7197
1,WOS:000234737300014,846,1,2006,10001
2,WOS:000241844400003,997,0,2006,14262
3,WOS:000417059300001,151,1,2006,21462
4,WOS:000235792300010,210,1,2006,15576
...,...,...,...,...,...
24339950,WOS:000750113400001,2719,0,2021,3115
24339951,WOS:000839344100002,1636,1,2022,7146
24339952,WOS:000798801800001,3584,0,2021,3478
24339953,WOS:000763296900001,2034,0,2021,13101


agg5에 edition 컬럼 생성 
기준 데이터 만들었으니 이제부터는 LEFT JOIN

In [25]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
edition  =  pd.read_csv('./dict/edition.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-15 17:43:28.983954
종료: 2024-02-15 17:44:15.031756


In [29]:
edition

,uid,edition
0,WOS:000201977400052,WOS.SCI
1,WOS:000200451400481,WOS.SCI
2,WOS:000205970500001,WOS.SSCI
3,WOS:000204963000018,WOS.SSCI
4,WOS:000200960000105,WOS.SCI
...,...,...
98689673,WOS:000883130700065,WOS.ESCI
98689674,WOS:000787659200001,WOS.ESCI
98689675,WOS:000815539500001,WOS.ESCI
98689676,WOS:000823752200014,WOS.ESCI


In [30]:
now = datetime.datetime.now() 
print('시작:', now)

# Save DataFrame to txt
result1 = pd.merge(agg5, edition, on='uid', how='left')

now = datetime.datetime.now()
print('종료:', now)


시작: 2024-02-15 17:54:02.382715
종료: 2024-02-15 17:55:52.875617


In [31]:
result1

,uid,cluster_id,core,pubyear2,n_clu,edition
0,WOS:000239031100003,1333,1,2006,7197,WOS.SCI
1,WOS:000239031100003,1333,1,2006,7197,WOS.SSCI
2,WOS:000234737300014,846,1,2006,10001,WOS.SCI
3,WOS:000241844400003,997,0,2006,14262,WOS.AHCI
4,WOS:000417059300001,151,1,2006,21462,WOS.SSCI
...,...,...,...,...,...,...
27362650,WOS:000750113400001,2719,0,2021,3115,WOS.SSCI
27362651,WOS:000839344100002,1636,1,2022,7146,WOS.SCI
27362652,WOS:000798801800001,3584,0,2021,3478,WOS.AHCI
27362653,WOS:000763296900001,2034,0,2021,13101,WOS.AHCI


In [32]:
result1.to_csv('./mid/first/result1.txt', sep='\t', index=False)

중간저장 후 로드

In [2]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
agg5 = pd.read_csv('./mid/agg5.txt', sep='\t')
edition =  pd.read_csv('./dict/edition.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-16 09:12:57.221637
종료: 2024-02-16 09:13:57.531251


In [3]:
# uid 컬럼의 고유한 값들의 수와 데이터 프레임의 행 수 비교
is_unique2 = len(edition['uid'].unique()) == len(edition)
print("UID 컬럼은 유일한가?", is_unique2)

UID 컬럼은 유일한가? False


In [4]:
agg5

,uid,cluster_id,core,pubyear2,n_clu
0,WOS:000239031100003,1333,1,2006,7197
1,WOS:000234737300014,846,1,2006,10001
2,WOS:000241844400003,997,0,2006,14262
3,WOS:000417059300001,151,1,2006,21462
4,WOS:000235792300010,210,1,2006,15576
...,...,...,...,...,...
24339950,WOS:000750113400001,2719,0,2021,3115
24339951,WOS:000839344100002,1636,1,2022,7146
24339952,WOS:000798801800001,3584,0,2021,3478
24339953,WOS:000763296900001,2034,0,2021,13101


In [5]:
edition

,uid,edition
0,WOS:000201977400052,WOS.SCI
1,WOS:000200451400481,WOS.SCI
2,WOS:000205970500001,WOS.SSCI
3,WOS:000204963000018,WOS.SSCI
4,WOS:000200960000105,WOS.SCI
...,...,...
98689673,WOS:000883130700065,WOS.ESCI
98689674,WOS:000787659200001,WOS.ESCI
98689675,WOS:000815539500001,WOS.ESCI
98689676,WOS:000823752200014,WOS.ESCI


In [6]:
# uid 컬럼에서 중복된 값을 찾음 (모든 중복 표시)
duplicated_rows = edition[edition.duplicated('uid', keep=False)]

# 중복된 uid 값들을 가진 행 출력
print(duplicated_rows)

duplicated_rows.to_csv('./check/duplicated_uids_edition.csv', index=False)

                          uid   edition
19        WOS:000202200400016   WOS.SCI
20        WOS:000202200400016  WOS.SSCI
25        WOS:000203534400002   WOS.SCI
26        WOS:000203534400002  WOS.SSCI
30        WOS:000206970300008   WOS.SCI
...                       ...       ...
98364477  WOS:000763024100001   WOS.SCI
98364478  WOS:000763024100001  WOS.SSCI
98364484  WOS:000853976200006   WOS.SCI
98364485  WOS:000853976200006  WOS.SSCI
98364486  WOS:000853976200006  WOS.AHCI

[21526960 rows x 2 columns]


복분류인경우 ;로 collapse

In [8]:
edition_collapsed = edition.groupby('uid')['edition'].apply(lambda x: ';'.join(x)).reset_index()

In [9]:
# uid 컬럼의 고유한 값들의 수와 데이터 프레임의 행 수 비교
is_unique3 = len(edition_collapsed['uid'].unique()) == len(edition_collapsed)
print("UID 컬럼은 유일한가?", is_unique3)

UID 컬럼은 유일한가? True


In [13]:
edition_collapsed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87694018 entries, 0 to 87694017
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   uid      object
 1   edition  object
dtypes: object(2)
memory usage: 1.3+ GB


In [14]:
edition_collapsed

,uid,edition
0,WOS:000003907500001,WOS.SCI
1,WOS:000003907500002,WOS.SCI
2,WOS:000003907500003,WOS.SCI
3,WOS:000003907500004,WOS.SCI
4,WOS:000003907500005,WOS.SCI
...,...,...
87694013,WOS:A1998YL52800034,WOS.SCI
87694014,WOS:A1998YL52800035,WOS.SCI
87694015,WOS:A1998YL55600001,WOS.SCI
87694016,WOS:A1998YL55600002,WOS.SCI


다시 조인해서 uid 유일성 체크 및 총 컬럼이 agg5

In [15]:
now = datetime.datetime.now() 
print('시작:', now)

# Save DataFrame to txt
result2 = pd.merge(agg5, edition_collapsed, on='uid', how='left')

now = datetime.datetime.now()
print('종료:', now)


시작: 2024-02-16 10:32:40.821913
종료: 2024-02-16 10:34:32.468353


In [16]:
result2

,uid,cluster_id,core,pubyear2,n_clu,edition
0,WOS:000239031100003,1333,1,2006,7197,WOS.SCI;WOS.SSCI
1,WOS:000234737300014,846,1,2006,10001,WOS.SCI
2,WOS:000241844400003,997,0,2006,14262,WOS.AHCI
3,WOS:000417059300001,151,1,2006,21462,WOS.SSCI
4,WOS:000235792300010,210,1,2006,15576,WOS.SCI
...,...,...,...,...,...,...
24339950,WOS:000750113400001,2719,0,2021,3115,WOS.SSCI
24339951,WOS:000839344100002,1636,1,2022,7146,WOS.SCI
24339952,WOS:000798801800001,3584,0,2021,3478,WOS.AHCI
24339953,WOS:000763296900001,2034,0,2021,13101,WOS.AHCI


In [18]:
result2.to_csv('./mid/first/result2.txt', sep='\t', index=False)

이어서 doctype 작업: 다양한 복분류 문서유형을 2개중 하나의 값으로 조정 'Ariticle', 'Review'

In [6]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
doctype =  pd.read_csv('./dict/doctype.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-16 15:17:25.636028
종료: 2024-02-16 15:18:11.136295


In [7]:
doctype

,uid,doctype2
0,WOS:000201977400052,Article
1,WOS:000200451400481,Article
2,WOS:000205970500001,Article
3,WOS:000204963000018,Book Review
4,WOS:000200960000105,Article
...,...,...
91995266,WOS:000883130700065,Article
91995267,WOS:000787659200001,Article
91995268,WOS:000815539500001,Editorial Material
91995269,WOS:000823752200014,Article


In [8]:
# uid 컬럼의 고유한 값들의 수와 데이터 프레임의 행 수 비교
is_unique4 = len(doctype['uid'].unique()) == len(doctype)
print("UID 컬럼은 유일한가?", is_unique4)

UID 컬럼은 유일한가? False


In [10]:
# uid 컬럼에서 중복된 값을 찾음 (모든 중복 표시)
duplicated_rows_doctype = doctype[doctype.duplicated('uid', keep=False)]

# 중복된 uid 값들을 가진 행 출력
print(duplicated_rows_doctype)

duplicated_rows_doctype .to_csv('./check/duplicated_rows_doctype.csv', index=False)

                          uid               doctype2
1849863   WOS:A1951UU81900001                Article
1849864   WOS:A1951UU81900001  Retracted Publication
7493115   WOS:A1974S943900044                 Letter
7493116   WOS:A1974S943900044  Retracted Publication
8170767   WOS:A1975AC24400005                Article
...                       ...                    ...
91995231  WOS:000842476500001           Early Access
91995237  WOS:000873774000001                Article
91995238  WOS:000873774000001           Early Access
91995254  WOS:000776099000001                Article
91995255  WOS:000776099000001           Early Access

[8602200 rows x 2 columns]


In [11]:
# uid 컬럼 기준으로 중복 제거 (첫 번째 행만 남김)
doctype_unique = doctype.drop_duplicates(subset='uid', keep='first')

# 결과 확인
print(doctype_unique)

doctype_unique.to_csv('./dict/doctype_unique.txt', sep='\t', index=False)


                          uid            doctype2
0         WOS:000201977400052             Article
1         WOS:000200451400481             Article
2         WOS:000205970500001             Article
3         WOS:000204963000018         Book Review
4         WOS:000200960000105             Article
...                       ...                 ...
91995266  WOS:000883130700065             Article
91995267  WOS:000787659200001             Article
91995268  WOS:000815539500001  Editorial Material
91995269  WOS:000823752200014             Article
91995270  WOS:000799950200001             Article

[87694018 rows x 2 columns]


In [12]:
# uid 컬럼의 고유한 값들의 수와 데이터 프레임의 행 수 비교
is_unique44 = len(doctype_unique['uid'].unique()) == len(doctype_unique)
print("UID 컬럼은 유일한가?", is_unique44)

UID 컬럼은 유일한가? True


In [13]:
# doctype DataFrame의 'doctype2' 컬럼에 포함된 고유한 값들과 그 빈도수를 확인
unique_values_counts_doctype2 = doctype_unique['doctype2'].value_counts()
print(unique_values_counts_doctype2)

doctype2
Article                           53685744
Meeting Abstract                   9057027
Proceedings Paper                  7392718
Book Review                        4678568
Editorial Material                 3786549
Review                             2590196
Letter                             2566797
Note                               1471070
News Item                           595096
Correction                          390153
Poetry                              300140
Biographical-Item                   154995
Correction, Addition                143778
Book                                130557
Art Exhibit Review                  128920
Item About an Individual             84005
Discussion                           80401
Record Review                        78351
Film Review                          72502
Music Performance Review             69340
Fiction, Creative Prose              50776
Theater Review                       36296
Dance Performance Review             26234
Mu

doctype이 'Article'과 'Review'인것만 뽑아서 rows수랑 유일성체크해보기

In [14]:
# 'Article'과 'Review'만 포함하는 행을 필터링
filtered_rows = doctype_unique[doctype_unique['doctype2'].isin(['Article', 'Review'])]

# 필터링된 행의 수
rows_count = filtered_rows.shape[0]

# 유일성 체크: 'Article'과 'Review'의 빈도수 확인
unique_values_counts_2col = filtered_rows['doctype2'].value_counts()

# 결과 출력
print(f"Rows count: {rows_count}")
print("Unique values counts:")
print(unique_values_counts_2col)

Rows count: 56275940
Unique values counts:
doctype2
Article    53685744
Review      2590196
Name: count, dtype: int64


In [15]:
filtered_rows

,uid,doctype2
0,WOS:000201977400052,Article
1,WOS:000200451400481,Article
2,WOS:000205970500001,Article
4,WOS:000200960000105,Article
5,WOS:000201487900019,Article
...,...,...
91995265,WOS:000742181300001,Article
91995266,WOS:000883130700065,Article
91995267,WOS:000787659200001,Article
91995269,WOS:000823752200014,Article


In [16]:
# uid 컬럼의 고유한 값들의 수와 데이터 프레임의 행 수 비교
is_unique5 = len(filtered_rows['uid'].unique()) == len(filtered_rows)
print("UID 컬럼은 유일한가?", is_unique5)

UID 컬럼은 유일한가? True


재로드

In [17]:
# left join


# reading 
now = datetime.datetime.now() 
print('시작:', now)
sorted_result7 =  pd.read_csv('./mid/first/sorted_result7.txt', sep='\t')
sorted2_result7 =  pd.read_csv('./mid/first/sorted2_result7.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)



시작: 2024-02-16 15:27:56.154025
종료: 2024-02-16 15:29:58.289477


In [18]:
sorted_result7

,uid,cluster_id,core,pubyear2,n_clu,edition,tc,source,source_abbrev,item,issn,eissn,doi_all
0,WOS:000237306100010,1,1,2006,52329,WOS.SCI,0.0,EPISODES,EPISODES,The IV International Symposium ProGEO on the C...,NaN,NaN,NaN
1,WOS:000245120500007,1,1,2006,52329,WOS.SCI,0.0,CANADIAN JOURNAL OF EARTH SCIENCES,CAN J EARTH SCI,Determination of alteration facies using the n...,0008-4077,NaN,10.1139/E06-097
2,WOS:000237229500002,1,1,2006,52329,WOS.SCI,0.0,INTERNATIONAL GEOLOGY REVIEW,INT GEOL REV,Sulfur isotope characteristics of mesothermal ...,0020-6814,NaN,10.2747/0020-6814.48.5.410
3,WOS:000241391900008,1,0,2006,52329,WOS.SCI,0.0,COMPTES RENDUS GEOSCIENCE,CR GEOSCI,Polish listeners to the mineralogy lectures gi...,1631-0713,NaN,10.1016/j.crte.2006.06.004
4,WOS:000241582600016,1,1,2006,52329,WOS.SCI,0.0,CANADIAN JOURNAL OF EARTH SCIENCES,CAN J EARTH SCI,Paleomagnetism of the wintering lake pluton an...,0008-4077,1480-3313,10.1139/E06-015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24339950,WOS:000592049600001,4215,1,2021,71,WOS.SCI,1.0,INTERNATIONAL JOURNAL OF SYSTEMS SCIENCE,INT J SYST SCI,Operator-based robust stability for nonlinear ...,NaN,NaN,NaN
24339951,WOS:000689390100001,4215,1,2021,71,WOS.SCI,1.0,MACHINES,MACHINES,Operator-Based Nonlinear Control of Calorimetr...,NaN,NaN,NaN
24339952,WOS:000741493600001,4215,1,2021,71,WOS.SCI,3.0,AXIOMS,AXIOMS,GPU Based Modelling and Analysis for Parallel ...,NaN,NaN,NaN
24339953,WOS:000661166500001,4215,1,2021,71,WOS.SCI,4.0,INTERNATIONAL JOURNAL OF ROBUST AND NONLINEAR ...,INT J ROBUST NONLIN,Robust passive tracking control for an uncerta...,NaN,NaN,NaN


In [19]:
sorted2_result7

,uid,cluster_id,core,pubyear2,n_clu,edition,tc,source,source_abbrev,item,issn,eissn,doi_all
0,WOS:000241489600007,1,1,2006,52329,WOS.SCI,2209.0,CHEMICAL GEOLOGY,CHEM GEOL,Hf isotopic compositions of the standard zirco...,0009-2541,1872-6836,10.1016/j.chemgeo.2006.05.003
1,WOS:000235430800004,1,1,2006,52329,WOS.SCI,1336.0,JOURNAL OF PETROLOGY,J PETROL,The genesis of intermediate and silicic magmas...,0022-3530,1460-2415,10.1093/petrology/egi084
2,WOS:000236201900004,1,1,2006,52329,WOS.SCI,1330.0,CONTRIBUTIONS TO MINERALOGY AND PETROLOGY,CONTRIB MINERAL PETR,Crystallization thermometers for zircon and ru...,0010-7999,1432-0967,10.1007/s00410-006-0068-5
3,WOS:000237306100004,1,1,2006,52329,WOS.SCI,1267.0,EPISODES,EPISODES,Petrogenesis of Mesozoic granitoids and volcan...,0705-3797,NaN,10.18814/epiiugs/2006/v29i1/004
4,WOS:000238242200001,1,1,2006,52329,WOS.SCI,1211.0,EARTH-SCIENCE REVIEWS,EARTH-SCI REV,Cenozoic tectonic evolution of the Himalayan o...,0012-8252,1872-6828,10.1016/j.earscirev.2005.05.004
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24339950,WOS:000661166500001,4215,1,2021,71,WOS.SCI,4.0,INTERNATIONAL JOURNAL OF ROBUST AND NONLINEAR ...,INT J ROBUST NONLIN,Robust passive tracking control for an uncerta...,NaN,NaN,NaN
24339951,WOS:000741493600001,4215,1,2021,71,WOS.SCI,3.0,AXIOMS,AXIOMS,GPU Based Modelling and Analysis for Parallel ...,NaN,NaN,NaN
24339952,WOS:000592049600001,4215,1,2021,71,WOS.SCI,1.0,INTERNATIONAL JOURNAL OF SYSTEMS SCIENCE,INT J SYST SCI,Operator-based robust stability for nonlinear ...,NaN,NaN,NaN
24339953,WOS:000689390100001,4215,1,2021,71,WOS.SCI,1.0,MACHINES,MACHINES,Operator-Based Nonlinear Control of Calorimetr...,NaN,NaN,NaN


In [20]:
result3 = pd.merge(sorted2_result7, filtered_rows, on='uid', how='left')

In [21]:
result3

,uid,cluster_id,core,pubyear2,n_clu,edition,tc,source,source_abbrev,item,issn,eissn,doi_all,doctype2
0,WOS:000241489600007,1,1,2006,52329,WOS.SCI,2209.0,CHEMICAL GEOLOGY,CHEM GEOL,Hf isotopic compositions of the standard zirco...,0009-2541,1872-6836,10.1016/j.chemgeo.2006.05.003,Article
1,WOS:000235430800004,1,1,2006,52329,WOS.SCI,1336.0,JOURNAL OF PETROLOGY,J PETROL,The genesis of intermediate and silicic magmas...,0022-3530,1460-2415,10.1093/petrology/egi084,Review
2,WOS:000236201900004,1,1,2006,52329,WOS.SCI,1330.0,CONTRIBUTIONS TO MINERALOGY AND PETROLOGY,CONTRIB MINERAL PETR,Crystallization thermometers for zircon and ru...,0010-7999,1432-0967,10.1007/s00410-006-0068-5,Article
3,WOS:000237306100004,1,1,2006,52329,WOS.SCI,1267.0,EPISODES,EPISODES,Petrogenesis of Mesozoic granitoids and volcan...,0705-3797,NaN,10.18814/epiiugs/2006/v29i1/004,Article
4,WOS:000238242200001,1,1,2006,52329,WOS.SCI,1211.0,EARTH-SCIENCE REVIEWS,EARTH-SCI REV,Cenozoic tectonic evolution of the Himalayan o...,0012-8252,1872-6828,10.1016/j.earscirev.2005.05.004,Review
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24339950,WOS:000661166500001,4215,1,2021,71,WOS.SCI,4.0,INTERNATIONAL JOURNAL OF ROBUST AND NONLINEAR ...,INT J ROBUST NONLIN,Robust passive tracking control for an uncerta...,NaN,NaN,NaN,Article
24339951,WOS:000741493600001,4215,1,2021,71,WOS.SCI,3.0,AXIOMS,AXIOMS,GPU Based Modelling and Analysis for Parallel ...,NaN,NaN,NaN,Article
24339952,WOS:000592049600001,4215,1,2021,71,WOS.SCI,1.0,INTERNATIONAL JOURNAL OF SYSTEMS SCIENCE,INT J SYST SCI,Operator-based robust stability for nonlinear ...,NaN,NaN,NaN,Article
24339953,WOS:000689390100001,4215,1,2021,71,WOS.SCI,1.0,MACHINES,MACHINES,Operator-Based Nonlinear Control of Calorimetr...,NaN,NaN,NaN,Article


In [22]:
result3.to_csv('./mid/first/result8.txt', sep='\t', index=False)
filtered_rows.to_csv('./mid/doctype_filtered_rows.txt', sep='\t', index=False)

최종본 저장을 위해 컬럼 순서 변경

In [23]:
# 컬럼 순서를 변경하고 싶은 순서대로 지정
new_order = ['uid' , 'cluster_id', 'core', 'n_clu', 'edition', 'pubyear2', 'doctype2', 'tc', 'source', 'source_abbrev', 'item', 'issn', 'eissn', 'doi_all']

# 새 순서로 DataFrame을 재구성
df_reordered = result3[new_order]

df_reordered.to_csv('./mid/fisrt/result9.txt', sep='\t', index=False)

tc 정수형처럼 표기 전처리

In [26]:
import numpy as np

In [32]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
result9 =  pd.read_csv('./mid/first/result9.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)


시작: 2024-02-16 15:57:31.179839
종료: 2024-02-16 15:58:44.142331


In [33]:
# 'tc' 컬럼을 pandas의 nullable 정수 데이터 타입으로 변환
result9['tc'] = result9['tc'].astype('Int64')

In [34]:
result9.to_csv('./mid/first/result10.txt', sep='\t', index=False)

tc 컬럼 만들기

In [30]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
tc =  pd.read_csv('./dict/coretc.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)




시작: 2024-02-16 15:55:39.398446
종료: 2024-02-16 15:56:22.117677


In [31]:
tc

,uid,tc
0,WOS:000003907500001,5
1,WOS:000003907500002,63
2,WOS:000003907500003,23
3,WOS:000003907500004,27
4,WOS:000003907500005,121
...,...,...
87723927,WOS:A1998YL52800034,0
87723928,WOS:A1998YL52800035,0
87723929,WOS:A1998YL55600001,6
87723930,WOS:A1998YL55600002,25


In [29]:
# uid 컬럼의 고유한 값들의 수와 데이터 프레임의 행 수 비교
is_unique6 = len(tc['uid'].unique()) == len(tc)
print("UID 컬럼은 유일한가?", is_unique6)

UID 컬럼은 유일한가? True


In [30]:
now = datetime.datetime.now() 
print('시작:', now)


result3 = pd.merge(result2, tc, on='uid', how='left')

now = datetime.datetime.now()
print('종료:', now)


시작: 2024-02-16 11:05:58.213766
종료: 2024-02-16 11:07:47.314401


In [31]:
result3

,uid,cluster_id,core,pubyear2,n_clu,edition,tc
0,WOS:000239031100003,1333,1,2006,7197,WOS.SCI;WOS.SSCI,33.0
1,WOS:000234737300014,846,1,2006,10001,WOS.SCI,85.0
2,WOS:000241844400003,997,0,2006,14262,WOS.AHCI,3.0
3,WOS:000417059300001,151,1,2006,21462,WOS.SSCI,68.0
4,WOS:000235792300010,210,1,2006,15576,WOS.SCI,131.0
...,...,...,...,...,...,...,...
24339950,WOS:000750113400001,2719,0,2021,3115,WOS.SSCI,2.0
24339951,WOS:000839344100002,1636,1,2022,7146,WOS.SCI,0.0
24339952,WOS:000798801800001,3584,0,2021,3478,WOS.AHCI,0.0
24339953,WOS:000763296900001,2034,0,2021,13101,WOS.AHCI,1.0


In [32]:
result3.to_csv('./mid/first/result3.txt', sep='\t', index=False)

source, source_abbrev컬럼 생성

In [33]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
source =  pd.read_csv('./dict/source.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-16 11:10:54.677609
종료: 2024-02-16 11:12:10.627815


In [34]:
# uid 컬럼의 고유한 값들의 수와 데이터 프레임의 행 수 비교
is_unique7 = len(source['uid'].unique()) == len(source)
print("UID 컬럼은 유일한가?", is_unique7)

UID 컬럼은 유일한가? False


In [35]:
source

,uid,source,source_abbrev
0,WOS:000201977400052,SCIENCE,SCIENCE
1,WOS:000200451400481,BRITISH MEDICAL JOURNAL,BRIT MED J
2,WOS:000205970500001,COLUMBIA LAW REVIEW,COLUMBIA LAW REV
3,WOS:000204963000018,PSYCHIATRIC QUARTERLY,PSYCHIAT QUART
4,WOS:000200960000105,COMPTES RENDUS DES SEANCES DE LA SOCIETE DE BI...,CR SOC BIOL
...,...,...,...
87694853,WOS:000883130700065,JOURNAL OF PHARMACY AND BIOALLIED SCIENCES,J PHARM BIOALLIED SC
87694854,WOS:000787659200001,THUNDERBIRD INTERNATIONAL BUSINESS REVIEW,THUNDERBIRD INT BUS
87694855,WOS:000815539500001,ECONOMISTS VOICE,ECON VOICE
87694856,WOS:000823752200014,ANNALES PHARMACEUTIQUES FRANCAISES,ANN PHARM FR


In [36]:
# uid 컬럼에서 중복된 값을 찾음 (모든 중복 표시)
duplicated_rows_source = source[source.duplicated('uid', keep=False)]

# 중복된 uid 값들을 가진 행 출력
print(duplicated_rows_source)

duplicated_rows_source.to_csv('./check/duplicated_rows_source.csv', index=False)

                          uid  \
46696683  WOS:000421182300002   
48515258  WOS:000421182300002   
61693676  WOS:000719414800004   
61715654  WOS:000782976900003   
61745272  WOS:000719414800015   
...                       ...   
84779360  WOS:000717519900024   
84779449  WOS:000717519900028   
84783731  WOS:000717630100067   
84793161  WOS:000717519900058   
84793315  WOS:000717630100005   

                                                     source  \
46696683                        NEW ZEALAND MEDICAL JOURNAL   
48515258                        NEW ZEALAND MEDICAL JOURNAL   
61693676  2015 IEEE 18TH INTERNATIONAL SYMPOSIUM ON DESI...   
61715654            ALBERTA JOURNAL OF EDUCATIONAL RESEARCH   
61745272  2015 IEEE 18TH INTERNATIONAL SYMPOSIUM ON DESI...   
...                                                     ...   
84779360  ADVANCES IN PRODUCTION MANAGEMENT SYSTEMS: ART...   
84779449  ADVANCES IN PRODUCTION MANAGEMENT SYSTEMS: ART...   
84783731  ADVANCES IN PRODUCTION MAN

중복행 전처리

In [39]:
# uid 컬럼 기준으로 중복 제거 (첫 번째 행만 남김)
source_unique = source.drop_duplicates(subset='uid', keep='first')

# 결과 확인
print(source_unique)

source_unique.to_csv('./dict/source_unique.txt', sep='\t', index=False)


                          uid  \
0         WOS:000201977400052   
1         WOS:000200451400481   
2         WOS:000205970500001   
3         WOS:000204963000018   
4         WOS:000200960000105   
...                       ...   
87694853  WOS:000883130700065   
87694854  WOS:000787659200001   
87694855  WOS:000815539500001   
87694856  WOS:000823752200014   
87694857  WOS:000799950200001   

                                                     source  \
0                                                   SCIENCE   
1                                   BRITISH MEDICAL JOURNAL   
2                                       COLUMBIA LAW REVIEW   
3                                     PSYCHIATRIC QUARTERLY   
4         COMPTES RENDUS DES SEANCES DE LA SOCIETE DE BI...   
...                                                     ...   
87694853         JOURNAL OF PHARMACY AND BIOALLIED SCIENCES   
87694854          THUNDERBIRD INTERNATIONAL BUSINESS REVIEW   
87694855                            

In [47]:

is_unique10 = len(source_unique ['uid'].unique()) == len(source_unique )
print("UID 컬럼은 유일한가?", is_unique10)


UID 컬럼은 유일한가? True


전처리 후 uid 유일성 확인 후 조인

In [48]:
now = datetime.datetime.now() 
print('시작:', now)

result5 = pd.merge(result3, source_unique, on='uid', how='left')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-16 13:36:01.665253
종료: 2024-02-16 13:37:42.789901


In [49]:
result5.to_csv('./mid/first/result5.txt', sep='\t', index=False)

In [50]:
result5 # 24339955

,uid,cluster_id,core,pubyear2,n_clu,edition,tc,source,source_abbrev
0,WOS:000239031100003,1333,1,2006,7197,WOS.SCI;WOS.SSCI,33.0,INTERNATIONAL JOURNAL OF GERIATRIC PSYCHIATRY,INT J GERIATR PSYCH
1,WOS:000234737300014,846,1,2006,10001,WOS.SCI,85.0,CLINICAL ENDOCRINOLOGY,CLIN ENDOCRINOL
2,WOS:000241844400003,997,0,2006,14262,WOS.AHCI,3.0,ZEITSCHRIFT FUR THEOLOGIE UND KIRCHE,Z THEOL KIRCHE
3,WOS:000417059300001,151,1,2006,21462,WOS.SSCI,68.0,POLITICS & GENDER,POLIT GENDER
4,WOS:000235792300010,210,1,2006,15576,WOS.SCI,131.0,BIOCHEMISTRY,BIOCHEMISTRY-US
...,...,...,...,...,...,...,...,...,...
24339950,WOS:000750113400001,2719,0,2021,3115,WOS.SSCI,2.0,ASIA PACIFIC LAW REVIEW,ASIA PAC LAW REV
24339951,WOS:000839344100002,1636,1,2022,7146,WOS.SCI,0.0,ONCOLOGY RESEARCH,ONCOL RES
24339952,WOS:000798801800001,3584,0,2021,3478,WOS.AHCI,0.0,EARLY POPULAR VISUAL CULTURE,EARLY POPUL VIS CULT
24339953,WOS:000763296900001,2034,0,2021,13101,WOS.AHCI,1.0,CAMBRIDGE OPERA JOURNAL,CAMB OPERA J


item 컬럼 생성

In [ ]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
title =  pd.read_csv('./dict/title.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)

UID 컬럼은 유일한가? False


In [42]:
title

,uid,item
0,WOS:000201977400052,A note on the ring method of measuring surface...
1,WOS:000200451400481,Chemical warfare and the doctor - II
2,WOS:000205970500001,NEW TRIALS FOR ERRONEOUS RULINGS UPON EVIDENCE...
3,WOS:000204963000018,Children's Behavior and Teachers' Attitudes
4,WOS:000200960000105,The collarette of choanocytes in heterocoelous...
...,...,...
87694853,WOS:000883130700065,Knowledge of rural population about orthodonti...
87694854,WOS:000787659200001,Globally distributed talent communities: A typ...
87694855,WOS:000815539500001,Editorial: Expansionary Monetary Policy and th...
87694856,WOS:000823752200014,Implementation of a multi-interventional appro...


title : uid 유일성 만족하게 전처리 작업

In [51]:
# uid 컬럼에서 중복된 값을 찾음 (모든 중복 표시)
duplicated_rows_title = title[title.duplicated('uid', keep=False)]

duplicated_rows_title.to_csv('./check/duplicated_rows_title.csv', index=False)

유일성 만족하면 left join

In [52]:
# uid 컬럼 기준으로 중복 제거 (첫 번째 행만 남김)
title_unique = title.drop_duplicates(subset='uid', keep='first')

is_unique9 = len(title_unique['uid'].unique()) == len(title_unique)
print("UID 컬럼은 유일한가?", is_unique9)

title_unique.to_csv('./dict/title_unique.txt', sep='\t', index=False)

UID 컬럼은 유일한가? True


In [53]:
now = datetime.datetime.now() 
print('시작:', now)

result6 = pd.merge(result5, title_unique, on='uid', how='left')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-16 13:46:15.934609
종료: 2024-02-16 13:47:57.957762


In [54]:
result6.to_csv('./mid/first/result6.txt', sep='\t', index=False)
is_unique11 = len(result6['uid'].unique()) == len(result6)
print("UID 컬럼은 유일한가?", is_unique11)


UID 컬럼은 유일한가? True


In [55]:
result6

,uid,cluster_id,core,pubyear2,n_clu,edition,tc,source,source_abbrev,item
0,WOS:000239031100003,1333,1,2006,7197,WOS.SCI;WOS.SSCI,33.0,INTERNATIONAL JOURNAL OF GERIATRIC PSYCHIATRY,INT J GERIATR PSYCH,Anemia and cognitive performance in hospitaliz...
1,WOS:000234737300014,846,1,2006,10001,WOS.SCI,85.0,CLINICAL ENDOCRINOLOGY,CLIN ENDOCRINOL,Increase of interferon-gamma-inducible CXC che...
2,WOS:000241844400003,997,0,2006,14262,WOS.AHCI,3.0,ZEITSCHRIFT FUR THEOLOGIE UND KIRCHE,Z THEOL KIRCHE,"Paul and Luther - A critical study of the ""New..."
3,WOS:000417059300001,151,1,2006,21462,WOS.SSCI,68.0,POLITICS & GENDER,POLIT GENDER,"Do Parties Know That ""Women Win""? Party Leader..."
4,WOS:000235792300010,210,1,2006,15576,WOS.SCI,131.0,BIOCHEMISTRY,BIOCHEMISTRY-US,Leveraging enzyme structure-function relations...
...,...,...,...,...,...,...,...,...,...,...
24339950,WOS:000750113400001,2719,0,2021,3115,WOS.SSCI,2.0,ASIA PACIFIC LAW REVIEW,ASIA PAC LAW REV,Transnational judicial dialogue in the rise of...
24339951,WOS:000839344100002,1636,1,2022,7146,WOS.SCI,0.0,ONCOLOGY RESEARCH,ONCOL RES,cGAS regulates the DNA damage response to main...
24339952,WOS:000798801800001,3584,0,2021,3478,WOS.AHCI,0.0,EARLY POPULAR VISUAL CULTURE,EARLY POPUL VIS CULT,'Cinema: Today's Theatre' - Images from the 19...
24339953,WOS:000763296900001,2034,0,2021,13101,WOS.AHCI,1.0,CAMBRIDGE OPERA JOURNAL,CAMB OPERA J,The Reproduction of Caruso


issn, eissn, doi_all 컬럼 생성

In [56]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
identifier =  pd.read_csv('./dict/identifier.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-16 13:50:20.952072
종료: 2024-02-16 13:50:47.056962


In [58]:
# uid 컬럼의 고유한 값들의 수와 데이터 프레임의 행 수 비교
is_unique12 = len(identifier['uid'].unique()) == len(identifier)
print("UID 컬럼은 유일한가?", is_unique12)

UID 컬럼은 유일한가? True


left join

In [59]:
now = datetime.datetime.now() 
print('시작:', now)

result7 = pd.merge(result6,identifier, on='uid', how='left')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-16 13:52:49.344418
종료: 2024-02-16 13:53:36.329084


In [61]:
result7.to_csv('./mid/first/result7.txt', sep='\t', index=False)

In [62]:
result7

,uid,cluster_id,core,pubyear2,n_clu,edition,tc,source,source_abbrev,item,issn,eissn,doi_all
0,WOS:000239031100003,1333,1,2006,7197,WOS.SCI;WOS.SSCI,33.0,INTERNATIONAL JOURNAL OF GERIATRIC PSYCHIATRY,INT J GERIATR PSYCH,Anemia and cognitive performance in hospitaliz...,0885-6230,NaN,10.1002/gps.1520
1,WOS:000234737300014,846,1,2006,10001,WOS.SCI,85.0,CLINICAL ENDOCRINOLOGY,CLIN ENDOCRINOL,Increase of interferon-gamma-inducible CXC che...,0300-0664,NaN,10.1111/j.1365-2265.2006.02447.x
2,WOS:000241844400003,997,0,2006,14262,WOS.AHCI,3.0,ZEITSCHRIFT FUR THEOLOGIE UND KIRCHE,Z THEOL KIRCHE,"Paul and Luther - A critical study of the ""New...",0044-3549,NaN,10.1628/004435406778655277
3,WOS:000417059300001,151,1,2006,21462,WOS.SSCI,68.0,POLITICS & GENDER,POLIT GENDER,"Do Parties Know That ""Women Win""? Party Leader...",NaN,NaN,NaN
4,WOS:000235792300010,210,1,2006,15576,WOS.SCI,131.0,BIOCHEMISTRY,BIOCHEMISTRY-US,Leveraging enzyme structure-function relations...,0006-2960,NaN,10.1021/bi052101l
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24339950,WOS:000750113400001,2719,0,2021,3115,WOS.SSCI,2.0,ASIA PACIFIC LAW REVIEW,ASIA PAC LAW REV,Transnational judicial dialogue in the rise of...,NaN,NaN,NaN
24339951,WOS:000839344100002,1636,1,2022,7146,WOS.SCI,0.0,ONCOLOGY RESEARCH,ONCOL RES,cGAS regulates the DNA damage response to main...,NaN,NaN,NaN
24339952,WOS:000798801800001,3584,0,2021,3478,WOS.AHCI,0.0,EARLY POPULAR VISUAL CULTURE,EARLY POPUL VIS CULT,'Cinema: Today's Theatre' - Images from the 19...,NaN,NaN,NaN
24339953,WOS:000763296900001,2034,0,2021,13101,WOS.AHCI,1.0,CAMBRIDGE OPERA JOURNAL,CAMB OPERA J,The Reproduction of Caruso,NaN,NaN,NaN


정렬시키기

tc 내림차순, 나머진 오름차순

In [74]:
sorted2_result7 = result7.sort_values(by=['cluster_id', 'pubyear2', 'tc'], ascending=[True, True, False])

In [75]:
sorted2_result7.to_csv('./mid/first/sorted2_result7.txt', sep='\t', index=False)

second dataset 만들기

In [65]:
# uid, cluster_id, pubyear2. item 
# 필요 컬럼만 추출

second_result1 = sorted_result7[['uid', 'cluster_id', 'pubyear2', 'item']]


In [66]:
second_result1.to_csv('./mid/second/second_result1.txt', sep='\t', index=False)

In [67]:
second_result1

,uid,cluster_id,pubyear2,item
11499,WOS:000237306100010,1,2006,The IV International Symposium ProGEO on the C...
16481,WOS:000245120500007,1,2006,Determination of alteration facies using the n...
51868,WOS:000237229500002,1,2006,Sulfur isotope characteristics of mesothermal ...
53519,WOS:000241391900008,1,2006,Polish listeners to the mineralogy lectures gi...
65974,WOS:000241582600016,1,2006,Paleomagnetism of the wintering lake pluton an...
...,...,...,...,...
20459101,WOS:000592049600001,4215,2021,Operator-based robust stability for nonlinear ...
23280582,WOS:000689390100001,4215,2021,Operator-Based Nonlinear Control of Calorimetr...
23967842,WOS:000741493600001,4215,2021,GPU Based Modelling and Analysis for Parallel ...
23778624,WOS:000661166500001,4215,2021,Robust passive tracking control for an uncerta...


aukey 컬럼 생성

In [68]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
aukey =  pd.read_csv('./mid/aukey.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-16 14:09:13.864935
종료: 2024-02-16 14:10:02.220876


In [70]:
aukey

,uid,keyword
0,WOS:000003907500002,biological control;evolution;invasion;specific...
1,WOS:000003907500003,beta-glucuronidase;integration vector;lucifera...
2,WOS:000003907500004,dones;DNA fingerprinting;population genetics;s...
3,WOS:000003907500005,cytochrome b gene;evolutionary history;intrasp...
4,WOS:000003907500006,cell-contact-dependent transfer;natural plasmi...
...,...,...
32940871,WOS:A1998YL41100011,straight-ahead direction;extra-retinal signals...
32940872,WOS:A1998YL41100012,dyslexia;reading;magnocellular;coherent motion
32940873,WOS:A1998YL55600001,electron spin echo;electron spin resonance;ele...
32940874,WOS:A1998YL55600002,NMR;imaging;contrast;solid-state;magnetization...


In [71]:
# uid 컬럼의 고유한 값들의 수와 데이터 프레임의 행 수 비교
is_unique13 = len(aukey['uid'].unique()) == len(aukey)
print("UID 컬럼은 유일한가?", is_unique13)

UID 컬럼은 유일한가? True


In [73]:
now = datetime.datetime.now() 
print('시작:', now)

second_result2= pd.merge(second_result1,aukey, on='uid', how='left')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-16 14:12:47.920579
종료: 2024-02-16 14:13:41.849717


In [76]:
second_result2

,uid,cluster_id,pubyear2,item,keyword
0,WOS:000237306100010,1,2006,The IV International Symposium ProGEO on the C...,NaN
1,WOS:000245120500007,1,2006,Determination of alteration facies using the n...,NaN
2,WOS:000237229500002,1,2006,Sulfur isotope characteristics of mesothermal ...,NaN
3,WOS:000241391900008,1,2006,Polish listeners to the mineralogy lectures gi...,history of geology;RJ Hauy;influence of French...
4,WOS:000241582600016,1,2006,Paleomagnetism of the wintering lake pluton an...,NaN
...,...,...,...,...,...
24339950,WOS:000592049600001,4215,2021,Operator-based robust stability for nonlinear ...,Nonlinear control;coprime factorisation;robust...
24339951,WOS:000689390100001,4215,2021,Operator-Based Nonlinear Control of Calorimetr...,operator theory;nonlinear control;right coprim...
24339952,WOS:000741493600001,4215,2021,GPU Based Modelling and Analysis for Parallel ...,fractional order derivative model;GPU;a spiral...
24339953,WOS:000661166500001,4215,2021,Robust passive tracking control for an uncerta...,integral sliding mode control;robust passive c...


In [77]:
second_result2.to_csv('./mid/second/second_result2.txt', sep='\t', index=False)

abstract 칼럼 생성

In [78]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
abstract =  pd.read_csv('./mid/abstract.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-16 14:24:59.835501
종료: 2024-02-16 14:34:15.553177


In [79]:
abstract

,uid,abstract
0,WOS:000208211300007,An explanation of the process of photographic ...
1,WOS:000208211700006,"Heat capacities, entropies, free energies, tot..."
2,WOS:000208211700003,A method of preparation of ammonias in which o...
3,WOS:000208211600005,In the July issue of Reviews of Modern Physics...
4,WOS:000208211400003,The lattice energies of NH4F and of the CsCl-t...
...,...,...
65650737,WOS:000823752200014,Background. - The impact of a multi-interventi...
65650738,Method. - This was a single-center interventio...,NaN
65650739,Results. - One hundred nine patients (mean age...,NaN
65650740,Conclusion. - The implementation of a MIMS in ...,NaN


In [80]:
# uid 컬럼의 고유한 값들의 수와 데이터 프레임의 행 수 비교
is_unique14 = len(abstract['uid'].unique()) == len(abstract)
print("UID 컬럼은 유일한가?", is_unique14)

UID 컬럼은 유일한가? False


유일성 맞게 전처리

In [81]:
# uid 컬럼에서 중복된 값을 찾음 (모든 중복 표시)
duplicated_rows_abstract= abstract[abstract.duplicated('uid', keep=False)]

duplicated_rows_abstract.to_csv('./check/duplicated_rows_abstract.csv', index=False)

In [85]:
# 용량이 너무 커서 부분만 저장 후 확인
duplicated_rows_abstract.head(100000).to_excel('./check/duplicated_rows_abstract_sub.xlsx', index=False)

In [86]:
# uid 컬럼 기준으로 중복 제거 (첫 번째 행만 남김)
abstract_unique = abstract.drop_duplicates(subset='uid', keep='first')


abstract_unique.to_csv('./mid/abstract_unique.txt', sep='\t', index=False)

In [87]:
is_unique15 = len(abstract_unique['uid'].unique()) == len(abstract_unique)
print("UID 컬럼은 유일한가?", is_unique15)

UID 컬럼은 유일한가? True


left join

In [3]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
second_result2 =  pd.read_csv('./mid/second/second_result2.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-16 16:28:42.287345
종료: 2024-02-16 16:29:41.849781


In [4]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
abstract_unique =  pd.read_csv('./mid/abstract_unique.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-16 16:29:41.854478
종료: 2024-02-16 16:39:03.473958


In [5]:
second_result2

,uid,cluster_id,pubyear2,item,keyword
0,WOS:000237306100010,1,2006,The IV International Symposium ProGEO on the C...,NaN
1,WOS:000245120500007,1,2006,Determination of alteration facies using the n...,NaN
2,WOS:000237229500002,1,2006,Sulfur isotope characteristics of mesothermal ...,NaN
3,WOS:000241391900008,1,2006,Polish listeners to the mineralogy lectures gi...,history of geology;RJ Hauy;influence of French...
4,WOS:000241582600016,1,2006,Paleomagnetism of the wintering lake pluton an...,NaN
...,...,...,...,...,...
24339950,WOS:000592049600001,4215,2021,Operator-based robust stability for nonlinear ...,Nonlinear control;coprime factorisation;robust...
24339951,WOS:000689390100001,4215,2021,Operator-Based Nonlinear Control of Calorimetr...,operator theory;nonlinear control;right coprim...
24339952,WOS:000741493600001,4215,2021,GPU Based Modelling and Analysis for Parallel ...,fractional order derivative model;GPU;a spiral...
24339953,WOS:000661166500001,4215,2021,Robust passive tracking control for an uncerta...,integral sliding mode control;robust passive c...


In [6]:
abstract_unique

,uid,abstract
0,WOS:000208211300007,An explanation of the process of photographic ...
1,WOS:000208211700006,"Heat capacities, entropies, free energies, tot..."
2,WOS:000208211700003,A method of preparation of ammonias in which o...
3,WOS:000208211600005,In the July issue of Reviews of Modern Physics...
4,WOS:000208211400003,The lattice energies of NH4F and of the CsCl-t...
...,...,...
64523106,WOS:000823752200014,Background. - The impact of a multi-interventi...
64523107,Method. - This was a single-center interventio...,NaN
64523108,Results. - One hundred nine patients (mean age...,NaN
64523109,Conclusion. - The implementation of a MIMS in ...,NaN


In [7]:
now = datetime.datetime.now() 
print('시작:', now)

second_result3 = pd.merge(second_result2,abstract_unique, on='uid', how='left')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-02-16 16:39:03.501114
종료: 2024-02-16 16:40:38.014573


In [8]:
# uid 컬럼의 고유한 값들의 수와 데이터 프레임의 행 수 비교
is_unique66 = len(second_result3['uid'].unique()) == len(second_result3)
print("UID 컬럼은 유일한가?", is_unique66)

UID 컬럼은 유일한가? True


In [9]:
second_result3.rename(columns={'keyword': 'aukey'}, inplace=True)

second_result3.to_csv('./mid/second/second_result3.txt', sep='\t', index=False)

In [10]:
second_result3

,uid,cluster_id,pubyear2,item,aukey,abstract
0,WOS:000237306100010,1,2006,The IV International Symposium ProGEO on the C...,NaN,NaN
1,WOS:000245120500007,1,2006,Determination of alteration facies using the n...,NaN,The Selbaie Cu-Zn deposit is located in the no...
2,WOS:000237229500002,1,2006,Sulfur isotope characteristics of mesothermal ...,NaN,Stable isotope analyses were carried out on su...
3,WOS:000241391900008,1,2006,Polish listeners to the mineralogy lectures gi...,history of geology;RJ Hauy;influence of French...,The paper analyses a list of thirty-four Poles...
4,WOS:000241582600016,1,2006,Paleomagnetism of the wintering lake pluton an...,NaN,This Lithoprobe-funded paleomagnetic study of ...
...,...,...,...,...,...,...
24339950,WOS:000592049600001,4215,2021,Operator-based robust stability for nonlinear ...,Nonlinear control;coprime factorisation;robust...,"In this paper, the robust stability of nonline..."
24339951,WOS:000689390100001,4215,2021,Operator-Based Nonlinear Control of Calorimetr...,operator theory;nonlinear control;right coprim...,"Recently, the development of SiC and GaN high-..."
24339952,WOS:000741493600001,4215,2021,GPU Based Modelling and Analysis for Parallel ...,fractional order derivative model;GPU;a spiral...,Heat exchangers are commonly used in various i...
24339953,WOS:000661166500001,4215,2021,Robust passive tracking control for an uncerta...,integral sliding mode control;robust passive c...,"As a research of human arm activities, the res..."


정렬 : cluster_id, pubyear2 둘다 오름차순

In [11]:
sorted_second_result3 = second_result3.sort_values(by=['cluster_id', 'pubyear2'], ascending=[True, True])


In [12]:
sorted_second_result3.to_csv('./mid/second/sorted_second_result3.txt', sep='\t', index=False)

In [13]:
sorted_second_result3

,uid,cluster_id,pubyear2,item,aukey,abstract
0,WOS:000237306100010,1,2006,The IV International Symposium ProGEO on the C...,NaN,NaN
1,WOS:000245120500007,1,2006,Determination of alteration facies using the n...,NaN,The Selbaie Cu-Zn deposit is located in the no...
2,WOS:000237229500002,1,2006,Sulfur isotope characteristics of mesothermal ...,NaN,Stable isotope analyses were carried out on su...
3,WOS:000241391900008,1,2006,Polish listeners to the mineralogy lectures gi...,history of geology;RJ Hauy;influence of French...,The paper analyses a list of thirty-four Poles...
4,WOS:000241582600016,1,2006,Paleomagnetism of the wintering lake pluton an...,NaN,This Lithoprobe-funded paleomagnetic study of ...
...,...,...,...,...,...,...
24339950,WOS:000592049600001,4215,2021,Operator-based robust stability for nonlinear ...,Nonlinear control;coprime factorisation;robust...,"In this paper, the robust stability of nonline..."
24339951,WOS:000689390100001,4215,2021,Operator-Based Nonlinear Control of Calorimetr...,operator theory;nonlinear control;right coprim...,"Recently, the development of SiC and GaN high-..."
24339952,WOS:000741493600001,4215,2021,GPU Based Modelling and Analysis for Parallel ...,fractional order derivative model;GPU;a spiral...,Heat exchangers are commonly used in various i...
24339953,WOS:000661166500001,4215,2021,Robust passive tracking control for an uncerta...,integral sliding mode control;robust passive c...,"As a research of human arm activities, the res..."


결측치 기록

In [14]:
#sorted_second_result3
# 각 컬럼별 null 값의 개수 계산
null_counts2 = sorted_second_result3.isnull().sum()

# 결과 출력
print(null_counts2)

uid                 0
cluster_id          0
pubyear2            0
item                0
aukey         6617219
abstract       786521
dtype: int64


None(Null) -> NA (txt에서는 빈 문자열로 저장되기 때문)

In [15]:
sorted_second_result3.to_csv('./final/second.txt', sep='\t', index=False, na_rep='NA')


## for check

In [ ]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
first =  pd.read_csv('./mid/first/final_result/final_first.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)

In [ ]:
# 각 컬럼별 null 값의 개수 계산
null_counts = first.isnull().sum()

# 결과 출력
print(null_counts)

None(Null) -> NA

In [ ]:
first.to_csv('./final/first.txt', sep='\t', index=False, na_rep='NA')